In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/pothole-image-segmentation-dataset'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

1.setup 

In [ ]:
# Install Ultralytics library
!pip install ultralytics

In [ ]:
# Disable warnings in the notebook to maintain clean output cells
import warnings
warnings.filterwarnings('ignore')

# Import necessary libraries
import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import cv2
import yaml
from PIL import Image
from collections import deque
from ultralytics import YOLO
from IPython.display import Video

In [ ]:
# Configure the visual appearance of Seaborn plots
sns.set(rc={'axes.facecolor': '#ffe4de'}, style='darkgrid')

**2.Load YOLOv8-seg Model**

In [ ]:
# Load the pre-trained YOLOv8 nano segmentation model
model = YOLO('yolov8n-seg.pt')

**3.Dataset Exploration**

In [ ]:
# Define the dataset_path
dataset_path = '/kaggle/input/pothole-image-segmentation-dataset'

# Set the path to the YAML file
yaml_file_path = os.path.join(dataset_path, '/kaggle/input/pothole-image-segmentation-dataset/Pothole_Segmentation_YOLOv8/data.yaml')

# Load and print the contents of the YAML file
with open(yaml_file_path, 'r') as file:
    yaml_content = yaml.load(file, Loader=yaml.FullLoader)
    print(yaml.dump(yaml_content, default_flow_style=False))

In [ ]:
# Set paths for training and validation image sets
train_images_path = os.path.join(dataset_path, '/kaggle/input/pothole-image-segmentation-dataset/Pothole_Segmentation_YOLOv8/train', 'images')
valid_images_path = os.path.join(dataset_path, '/kaggle/input/pothole-image-segmentation-dataset/Pothole_Segmentation_YOLOv8/valid', 'images')

# Initialize counters for the number of images
num_train_images = 0
num_valid_images = 0

# Initialize sets to hold the unique sizes of images
train_image_sizes = set()
valid_image_sizes = set()

# Check train images sizes and count
for filename in os.listdir(train_images_path):
    if filename.endswith('.jpg'):  
        num_train_images += 1
        image_path = os.path.join(train_images_path, filename)
        with Image.open(image_path) as img:
            train_image_sizes.add(img.size)

# Check validation images sizes and count
for filename in os.listdir(valid_images_path):
    if filename.endswith('.jpg'): 
        num_valid_images += 1
        image_path = os.path.join(valid_images_path, filename)
        with Image.open(image_path) as img:
            valid_image_sizes.add(img.size)

# Print the results
print(f"Number of training images: {num_train_images}")
print(f"Number of validation images: {num_valid_images}")

# Check if all images in training set have the same size
if len(train_image_sizes) == 1:
    print(f"All training images have the same size: {train_image_sizes.pop()}")
else:
    print("Training images have varying sizes.")

# Check if all images in validation set have the same size
if len(valid_image_sizes) == 1:
    print(f"All validation images have the same size: {valid_image_sizes.pop()}")
else:
    print("Validation images have varying sizes.")

In [ ]:
# Set the seed for the random number generator
random.seed(0)

# Create a list of image files
image_files = [f for f in os.listdir(train_images_path) if f.endswith('.jpg')]

# Randomly select 15 images
random_images = random.sample(image_files, 15)

# Create a new figure
plt.figure(figsize=(19, 12))

# Loop through each image and display it in a 3x5 grid
for i, image_file in enumerate(random_images):
    image_path = os.path.join(train_images_path, image_file)
    image = Image.open(image_path)
    plt.subplot(3, 5, i + 1)
    plt.imshow(image)
    plt.axis('off')
# Add a suptitle
plt.suptitle('Random Selection of Dataset Images', fontsize=24)

# Show the plot
plt.tight_layout()
plt.show()

# Deleting unnecessary variable to free up memory
del image_files

**4.Fine-tuning yolov8n**

In [ ]:
import torch
print("CUDA Available:", torch.cuda.is_available())
print("CUDA Device Count:", torch.cuda.device_count())
print("CUDA Version:", torch.version.cuda)
print("Device Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")


In [ ]:
import torch
device = 0 if torch.cuda.is_available() else "cpu"  # Auto-select GPU if available

results = model.train(
    data=yaml_file_path,
    epochs=15,
    imgsz=640,
    patience=15,
    batch=16,
    optimizer='auto',
    lr0=0.0001,
    lrf=0.01,
    dropout=0.25,
    device=device,  # Auto GPU/CPU selection
    seed=42
)


**5.Model Performance Evaluation**

In [ ]:
# Define the path to the directory
post_training_files_path = '/kaggle/working/runs/segment/train'

# List the files in the directory
!ls {post_training_files_path}

Validation performance metric evaluation

In [ ]:
# Construct the path to the best model weights file using os.path.join
best_model_path = os.path.join(post_training_files_path, 'weights/best.pt')

# Load the best model weights into the YOLO model
best_model = YOLO(best_model_path)

# Validate the best model using the validation set with default parameters
metrics = best_model.val(split='val')

In [ ]:
# Convert the dictionary to a pandas DataFrame and use the keys as the index
metrics_df = pd.DataFrame.from_dict(metrics.results_dict, orient='index', columns=['Metric Value'])

# Display the DataFrame
metrics_df.round(3)

**Model testing**

****6.1 Test on validation image**###### **

In [ ]:
# Define the path to the validation images
valid_images_path = os.path.join(dataset_path, '/kaggle/input/pothole-image-segmentation-dataset/Pothole_Segmentation_YOLOv8/valid', 'images')

# List all jpg images in the directory
image_files = [file for file in os.listdir(valid_images_path) if file.endswith('.jpg')]

# Select 9 images at equal intervals
num_images = len(image_files)
selected_images = [image_files[i] for i in range(0, num_images, num_images // 9)]

# Initialize the subplot
fig, axes = plt.subplots(3, 3, figsize=(20, 21))
fig.suptitle('Validation Set Inferences', fontsize=24)

# Perform inference on each selected image and display it
for i, ax in enumerate(axes.flatten()):
    image_path = os.path.join(valid_images_path, selected_images[i])
    results = best_model.predict(source=image_path, imgsz=640)
    annotated_image = results[0].plot()
    annotated_image_rgb = cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB)
    ax.imshow(annotated_image_rgb)
    ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Define the path to the sample video in the dataset
dataset_video_path = '/kaggle/input/pothole-image-segmentation-dataset/Pothole_Segmentation_YOLOv8/sample_video.mp4'

# Define the destination path in the working directory
video_path = '/kaggle/working/sample_video.mp4'

# Copy the video file from its original location in the dataset to the current working directory in Kaggle
shutil.copyfile(dataset_video_path, video_path)

# Initiate vehicle detection on the sample video using the best performing model and save the output
best_model.predict(source=video_path, save=True)

In [ ]:
# Convert the .avi video generated by the YOLOv8 prediction to .mp4 format for compatibility with notebook display
!ffmpeg -y -loglevel panic -i /kaggle/working/runs/segment/predict/sample_video.avi processed_sample_video.mp4

# Embed and display the processed sample video within the notebook
Video("processed_sample_video.mp4", embed=True, width=960)

In [ ]:
Save the model

In [ ]:
# Export the model
best_model.export(format='onnx')


**Realtime Road damage assessment**

In [ ]:
# Define the path to the validation images
valid_images_path = os.path.join(dataset_path, '/kaggle/input/pothole-image-segmentation-dataset/Pothole_Segmentation_YOLOv8/valid', 'images')

# List all jpg images in the directory
image_files = [file for file in os.listdir(valid_images_path) if file.endswith('.jpg')]

# Select a sample image
selected_image = image_files[30]

# Perform inference on the selected image
image_path = os.path.join(valid_images_path, selected_image)
results = best_model.predict(source=image_path, imgsz=640, conf=0.5)
annotated_image = results[0].plot()
annotated_image_rgb = cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB)

# Determine the number of subplots needed (1 original + number of masks)
num_subplots = 1 + (len(results[0].masks.data) if results[0].masks is not None else 0)

# Initialize the subplot with 1 row and n columns
fig, axes = plt.subplots(1, num_subplots, figsize=(15, 5))

# Display the original annotated image
axes[0].imshow(annotated_image_rgb)
axes[0].set_title('Original Image')
axes[0].axis('off')

# If multiple masks, iterate and display each mask
if results[0].masks is not None:
    masks = results[0].masks.data.cpu().numpy()
    for i, mask in enumerate(masks):
        # Threshold the mask to make sure it's binary
        # Any value greater than 0 is set to 255, else it remains 0
        binary_mask = (mask > 0).astype(np.uint8) * 255
        axes[i+1].imshow(binary_mask, cmap='gray')
        axes[i+1].set_title(f'Segmented Mask {i+1}')
        axes[i+1].axis('off')

# Adjust layout and display the subplot
plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

# Initialize variables to hold total area and individual areas
total_area = 0
area_list = []

# Perform operations if masks are available
if results[0].masks is not None:
    masks = results[0].masks.data.cpu().numpy()   # Retrieve masks as numpy arrays
    image_area = masks.shape[1] * masks.shape[2]  # Calculate total number of pixels in the image
    
    # Set up the subplot dynamically based on the number of masks
    fig, axes = plt.subplots(1, len(masks), figsize=(12, 8))
    
    # Ensure `axes` is iterable even if there is only one mask
    if len(masks) == 1:
        axes = [axes]

    for i, mask in enumerate(masks):
        binary_mask = (mask > 0).astype(np.uint8) * 255  # Convert mask to binary
        color_mask = cv2.cvtColor(binary_mask, cv2.COLOR_GRAY2BGR)  # Convert binary mask to color
        contours, _ = cv2.findContours(binary_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        
        if contours:  # Ensure that at least one contour is found
            contour = contours[0]  # Retrieve the first contour
            area = cv2.contourArea(contour)  # Calculate the area of the pothole
            area_list.append(area)  # Append area to the list
            cv2.drawContours(color_mask, [contour], -1, (0, 255, 0), 3)  # Draw the contour on the mask

            # Display the mask with the green contour
            axes[i].imshow(color_mask)
            axes[i].set_title(f'Pothole {i+1}')
            axes[i].axis('off')

    # Display all masks
    plt.tight_layout()
    plt.show()

    # Calculate and print areas after displaying the images
    for i, area in enumerate(area_list):
        print(f"Area of Pothole {i+1}: {area} pixels")  
        total_area += area  # Sum the areas for total

    # Calculate and print the total damaged area and percentage of road damaged by potholes
    print("-"*50)
    print(f"Total Damaged Area by Potholes: {total_area} pixels")
    print(f"Total Pixels in Image: {image_area} pixels")
    print(f"Percentage of Road Damaged: {(total_area / image_area) * 100:.2f}%")


In [ ]:
# Define the video path
video_path = '/kaggle/working/sample_video.mp4'

# Define font, scale, colors, and position for the annotation
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 1
text_position = (40, 80)
font_color = (255, 255, 255)    # White color for text
background_color = (0, 0, 255)  # Red background for text

# Initialize a deque with fixed length for averaging the last 10 percentage damages
damage_deque = deque(maxlen=10)

# Open the video
cap = cv2.VideoCapture(video_path)

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('road_damage_assessment.avi', fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

# Read until video is completed
while cap.isOpened():
     # Capture frame-by-frame
    ret, frame = cap.read()
    if ret:
        # Perform inference on the frame
        results = best_model.predict(source=frame, imgsz=640, conf=0.25)
        processed_frame = results[0].plot(boxes=False)
        
        # Initializes percentage_damage to 0
        percentage_damage = 0 

         # If masks are available, calculate total damage area and percentage
        if results[0].masks is not None:
            total_area = 0
            masks = results[0].masks.data.cpu().numpy()
            image_area = frame.shape[0] * frame.shape[1]  # total number of pixels in the image
            for mask in masks:
                binary_mask = (mask > 0).astype(np.uint8) * 255
                contour, _ = cv2.findContours(binary_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
                total_area += cv2.contourArea(contour[0])
            
            percentage_damage = (total_area / image_area) * 100

        # Calculate and update the percentage damage
        damage_deque.append(percentage_damage)
        smoothed_percentage_damage = sum(damage_deque) / len(damage_deque)
            
        # Draw a thick line for text background
        cv2.line(processed_frame, (text_position[0], text_position[1] - 10),
                 (text_position[0] + 350, text_position[1] - 10), background_color, 40)
        # Annotate the frame with the percentage of damage
        cv2.putText(processed_frame, f'Road Damage: {smoothed_percentage_damage:.2f}%', text_position, font, font_scale, font_color, 2, cv2.LINE_AA)         
    
        # Write the processed frame to the output video
        out.write(processed_frame)
        
        # Uncomment the following 3 lines if running this code on a local machine to view the real-time processing results
        # cv2.imshow('Road Damage Assessment', processed_frame) # Display the processed frame
        # if cv2.waitKey(1) & 0xFF == ord('q'): # Press Q on keyboard to exit the loop
        #     break 
    else:
        break

# Release the video capture and video write objects
cap.release()
out.release()

# Close all the frames
# cv2.destroyAllWindows()


In [ ]:
# Convert the .avi video generated by our traffic density estimation app to .mp4 format for compatibility with notebook display
!ffmpeg -y -loglevel panic -i /kaggle/working/road_damage_assessment.avi road_damage_assessment.mp4

# Embed and display the processed sample video within the notebook
Video("road_damage_assessment.mp4", embed=True, width=960)

In [ ]:
!pip install -q gradio onnxruntime


In [ ]:
import os
print(os.listdir("/kaggle/input"))


In [ ]:
import cv2
import numpy as np
import os
import torch
import gradio as gr
import matplotlib.pyplot as plt
from ultralytics import YOLO
from collections import deque

# ✅ Load YOLOv8 model
model_path = "/kaggle/working/runs/segment/train/weights/best.pt"
model = YOLO(model_path)

# ✅ Function to process images and analyze damage
def analyze_damage(image):
    orig_image = image.copy()
    h, w, _ = orig_image.shape

    # Perform inference
    results = model(image)

    # Draw detections
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
            score = box.conf[0].item()
            if score > 0.5:
                cv2.rectangle(orig_image, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(orig_image, f"Pothole {score:.2f}", (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    # ✅ Calculate total damage area
    total_area = 0
    image_area = h * w  # Total image pixels

    if hasattr(results[0], 'masks') and results[0].masks is not None:
        masks = results[0].masks.data.cpu().numpy()
        for mask in masks:
            binary_mask = (mask > 0).astype(np.uint8) * 255
            contours, _ = cv2.findContours(binary_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            if contours:
                total_area += cv2.contourArea(contours[0])

    damage_percentage = (total_area / image_area) * 100 if image_area > 0 else 0

    return orig_image, f"🚧 Road Damage: {damage_percentage:.2f}% 🚧"

# ✅ Function to process videos
def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter('/kaggle/working/road_damage_output.avi', fourcc, 20.0,
                          (int(cap.get(3)), int(cap.get(4))))
    
    damage_deque = deque(maxlen=10)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Run inference
        results = model(frame)
        processed_frame = results[0].plot(boxes=False)

        # Calculate road damage
        total_area = 0
        image_area = frame.shape[0] * frame.shape[1]

        if results[0].masks is not None:
            masks = results[0].masks.data.cpu().numpy()
            for mask in masks:
                binary_mask = (mask > 0).astype(np.uint8) * 255
                contours, _ = cv2.findContours(binary_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
                if contours:
                    total_area += cv2.contourArea(contours[0])

        damage_percentage = (total_area / image_area) * 100 if image_area > 0 else 0
        damage_deque.append(damage_percentage)
        smoothed_damage = sum(damage_deque) / len(damage_deque)

        # Overlay damage percentage
        cv2.putText(processed_frame, f'Road Damage: {smoothed_damage:.2f}%', (40, 80),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        out.write(processed_frame)

    cap.release()
    out.release()
    return "/kaggle/working/road_damage_output.avi"

# 🎨 **Enhanced Gradio UI**
with gr.Blocks(title="Road Damage Assessment") as app:
    gr.Markdown("<h1 style='text-align: center; color: #4CAF50;'>🚧 Road Damage Detection & Assessment 🚧</h1>")
    gr.Markdown("### Upload an image or video to analyze road damage and pothole severity.")

    with gr.Tab("📸 Image Analysis"):
        with gr.Row():
            input_image = gr.Image(type="numpy", label="Upload Road Image")
            analyze_button = gr.Button("🔍 Analyze Damage")
        with gr.Row():
            output_image = gr.Image(type="numpy", label="Damage Analysis Output")
            damage_text = gr.Textbox(label="Damage Percentage", interactive=False)
        analyze_button.click(analyze_damage, inputs=input_image, outputs=[output_image, damage_text])

    with gr.Tab("🎥 Video Analysis"):
        with gr.Row():
            input_video = gr.Video(label="Upload Road Video")
            analyze_video_button = gr.Button("🎬 Process Video")
        with gr.Row():
            output_video = gr.Video(label="Processed Video Output")
        analyze_video_button.click(process_video, inputs=input_video, outputs=output_video)

# 🚀 Launch UI
app.launch()


In [ ]:
import cv2
import numpy as np
import os
import torch
import gradio as gr
import matplotlib.pyplot as plt
from ultralytics import YOLO
from collections import deque

# ✅ Load YOLOv8 model
model_path = "/kaggle/working/runs/segment/train/weights/best.pt"
model = YOLO(model_path)

# ✅ Function to process images and analyze damage
def analyze_damage(image):
    orig_image = image.copy()
    h, w, _ = orig_image.shape

    # Perform inference
    results = model(image)

    # Draw detections
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
            score = box.conf[0].item()
            if score > 0.5:
                cv2.rectangle(orig_image, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(orig_image, f"Pothole {score:.2f}", (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    # ✅ Calculate total damage area
    total_area = 0
    image_area = h * w  # Total image pixels

    if hasattr(results[0], 'masks') and results[0].masks is not None:
        masks = results[0].masks.data.cpu().numpy()
        for mask in masks:
            binary_mask = (mask > 0).astype(np.uint8) * 255
            contours, _ = cv2.findContours(binary_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            if contours:
                total_area += cv2.contourArea(contours[0])

    damage_percentage = (total_area / image_area) * 100 if image_area > 0 else 0

    return orig_image, f"🚧 Road Damage: {damage_percentage:.2f}% 🚧"

# ✅ Function to process videos
def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter('/kaggle/working/road_damage_output.avi', fourcc, 20.0,
                          (int(cap.get(3)), int(cap.get(4))))
    
    damage_deque = deque(maxlen=10)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Run inference
        results = model(frame)
        processed_frame = results[0].plot(boxes=False)

        # Calculate road damage
        total_area = 0
        image_area = frame.shape[0] * frame.shape[1]

        if results[0].masks is not None:
            masks = results[0].masks.data.cpu().numpy()
            for mask in masks:
                binary_mask = (mask > 0).astype(np.uint8) * 255
                contours, _ = cv2.findContours(binary_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
                if contours:
                    total_area += cv2.contourArea(contours[0])

        damage_percentage = (total_area / image_area) * 100 if image_area > 0 else 0
        damage_deque.append(damage_percentage)
        smoothed_damage = sum(damage_deque) / len(damage_deque)

        # Overlay damage percentage
        cv2.putText(processed_frame, f'Road Damage: {smoothed_damage:.2f}%', (40, 80),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        out.write(processed_frame)

    cap.release()
    out.release()
    return "/kaggle/working/road_damage_output.avi"

# 🎨 **Enhanced Gradio UI**
with gr.Blocks(title="Road Damage Assessment") as app:
    gr.Markdown("""
        # 🚧 Road Damage Detection & Assessment 🚧
        Upload an image or video to analyze road damage and pothole severity.
    """)

    with gr.Tab("📸 Image Analysis"):
        gr.Markdown("### Upload an image to detect potholes and assess damage severity.")
        with gr.Row():
            input_image = gr.Image(type="numpy", label="Upload Road Image")
            analyze_button = gr.Button("🔍 Analyze Damage", variant="primary")
        with gr.Row():
            output_image = gr.Image(type="numpy", label="Damage Analysis Output")
            damage_text = gr.Textbox(label="Damage Percentage", interactive=False)
        analyze_button.click(analyze_damage, inputs=input_image, outputs=[output_image, damage_text])

    with gr.Tab("🎥 Video Analysis"):
        gr.Markdown("### Upload a video for real-time road damage detection.")
        with gr.Row():
            input_video = gr.Video(label="Upload Road Video")
            analyze_video_button = gr.Button("🎬 Process Video", variant="primary")
        with gr.Row():
            output_video = gr.Video(label="Processed Video Output")
        analyze_video_button.click(process_video, inputs=input_video, outputs=output_video)

# 🚀 Launch UI
app.launch()